## Links

https://python.langchain.com/docs/modules/agents/

## Required Imports

In [64]:
import os
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from langchain.globals import set_verbose, set_debug

debug=False
set_verbose(debug)
set_debug(debug)

OpenAPIKey = os.environ.get("OPENAI_API_KEY")


## Simple Tool Test

This is excluding the runtime, hence it will resolve the function call and the arguments but not actually call the function (called tool in langchain).

In [18]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = model.bind(functions=[format_tool_to_openai_function(t) for t in tools])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent.invoke({"input": "how many letters in the word cookie worm?", "intermediate_steps": []})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'cookie worm'}, log="\nInvoking: `get_word_length` with `{'word': 'cookie worm'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "cookie worm"\n}', 'name': 'get_word_length'}})])

## Define the Runtime

This will drive the completion of the task.

In [19]:
from langchain.schema.agent import AgentFinish

user_input = "how many letters in the word cookie worm?"
intermediate_steps = []
while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = {"get_word_length": get_word_length}[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

TOOL NAME: get_word_length
TOOL INPUT: {'word': 'cookie worm'}
There are 11 letters in the word "cookie worm".


## Use a Built-in Executor: AgentExecutor

In [20]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=debug)

agent_executor.invoke({"input": "how many letters in the word cookie worm?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'cookie worm'}`


11There are 11 letters in the word "cookie worm".

> Finished chain.


{'input': 'how many letters in the word cookie worm?',
 'output': 'There are 11 letters in the word "cookie worm".'}

## Add Memory to the Agent

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=debug)

# First call
input = "how many letters in the word cookie worm?"
result = agent_executor.invoke({"input": input, "chat_history": chat_history})

# Add messages to chat history
chat_history.extend(
    [
        HumanMessage(content=input),
        AIMessage(content=result["output"]),
    ]
)

# Second call
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})

## Get all Built-in Tools

In [ ]:
from langchain.agents import get_all_tool_names
get_all_tool_names()


## My First Custom Tool (SMHI)

In [77]:
import requests
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Optional, Type


class ForecastInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

class ForecastOutput(BaseModel):
    temperature: Optional[float] = Field(None, description="Temperature in degrees Celsius")
    wind_speed: Optional[float] = Field(None, description="Wind speed in meters per second")
    precipitation: Optional[float] = Field(None, description="Precipitation in millimeters")

class ForecastTool(StructuredTool):
    args_schema: Type[BaseModel] = ForecastInput
    return_schema: Type[BaseModel] = ForecastOutput
    name: str = "forecast"
    description: str = "Useful when you need to answer a question about weather in a specific location."

    def _run(self, input: ForecastInput) -> ForecastOutput:
        # SMHI API endpoint for weather forecast
        url = f"https://opendata-download-metfcst.smhi.se/api/category/pmp3g/version/2/geotype/point/lon/{input.longitude}/lat/{input.latitude}/data.json"

        print(f"Retrieving weather forecast for lat: {input.latitude}, lon: {input.longitude}")
        response = requests.get(url)

        if response.status_code == 200:
            return self.extract_weather_info(response=response)
        else:
            raise Exception("Error: Could not retrieve the weather forecast.")

    def extract_weather_info(self, response: requests.Response) -> ForecastOutput:
        forecast=response.json()
        
        if 'timeSeries' in forecast and len(forecast['timeSeries']) > 0:
            # The first element in the time series is usually the current weather forecast
            current_forecast = forecast['timeSeries'][0]

            weather_info: ForecastOutput = {
                'temperature': None,
                'wind_speed': None,
                'precipitation': None
            }

            for parameter in current_forecast['parameters']:
                if parameter['name'] == 't':  # Temperature
                    weather_info['temperature'] = parameter['values'][0]
                elif parameter['name'] == 'ws':  # Wind speed
                    weather_info['wind_speed'] = parameter['values'][0]
                elif parameter['name'] == 'pmean':  # Mean precipitation
                    weather_info['precipitation'] = parameter['values'][0]

            return weather_info
        else:
            raise Exception("Error: Could not parse the weather forecast.")


# Replace with the coordinates of your city in Sweden
latitude = "59.3293"  # Example: Stockholm
longitude = "18.0686"

forecast_tool = ForecastTool()
forecast_tool._run(ForecastInput(latitude=latitude, longitude=longitude))

Retrieving weather forecast for lat: 59.3293, lon: 18.0686


{'temperature': -0.8, 'wind_speed': 6.6, 'precipitation': 0.1}

## Use SMHI Tool in Agent

In [78]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are very powerful assistant, but you need to use a tool get weather info."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=[ForecastTool()], verbose=True)

result = agent_executor.invoke({"input": "how cold is it in Skellefteå?"})

print(result)



> Entering new AgentExecutor chain...
To get the current weather information for Skellefteå, I can use a weather API. However, as a text-based AI, I don't have direct access to real-time data. I suggest you check a reliable weather website or use a weather app on your device to get the most accurate and up-to-date information about the temperature in Skellefteå.

> Finished chain.
{'input': 'how cold is it in Skellefteå?', 'output': "To get the current weather information for Skellefteå, I can use a weather API. However, as a text-based AI, I don't have direct access to real-time data. I suggest you check a reliable weather website or use a weather app on your device to get the most accurate and up-to-date information about the temperature in Skellefteå."}
